# Auto ML for Images

This notebook will show you step by step how to train and deploy an AutoML for images model. 

We will cover data preparation, model training, evaluation, and deployment. Let's get started!

This notebook is based on this [Tutorial: Train an object detection model with AutoML and Python](https://learn.microsoft.com/en-au/azure/machine-learning/tutorial-auto-train-image-models?view=azureml-api-2&tabs=python).

## Experiment setup

Use Azure Machine learning to track experiments

In [ ]:
exp_name = "coffee-cup-detection"

## Visualization

In [ ]:
# %pip install --upgrade matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image as pil_image
import numpy as np
import json
import os

def plot_ground_truth_boxes(image_file, ground_truth_boxes):
    # Display the image
    plt.figure()
    img_np = mpimg.imread(image_file)
    img = pil_image.fromarray(img_np.astype("uint8"), "RGB")
    img_w, img_h = img.size

    fig,ax = plt.subplots(figsize=(12, 16))
    ax.imshow(img_np)
    ax.axis("off")

    label_to_color_mapping = {}

    print(ground_truth_boxes)

    for gt in ground_truth_boxes:
        label = gt["label"]

        xmin, ymin, xmax, ymax =  gt["topX"], gt["topY"], gt["bottomX"], gt["bottomY"]
        topleft_x, topleft_y = img_w * xmin, img_h * ymin
        width, height = img_w * (xmax - xmin), img_h * (ymax - ymin)

        if label in label_to_color_mapping:
            color = label_to_color_mapping[label]
        else:
            # Generate a random color. If you want to use a specific color, you can use something like "red".
            color = np.random.rand(3)
            label_to_color_mapping[label] = color

        # Display bounding box
        rect = patches.Rectangle((topleft_x, topleft_y), width, height,
                                 linewidth=2, edgecolor=color, facecolor="none")
        ax.add_patch(rect)

        # Display label
        ax.text(topleft_x, topleft_y - 10, label, color=color, fontsize=20)

    plt.show()

def plot_ground_truth_boxes_jsonl(image_file, jsonl_file):
    image_base_name = os.path.basename(image_file)
    ground_truth_data_found = False
    with open(jsonl_file) as fp:
        for line in fp.readlines():
            line_json = json.loads(line)
            filename = line_json["image_url"]
            if image_base_name in filename:
                ground_truth_data_found = True
                plot_ground_truth_boxes(image_file, line_json["label"])
                break
    if not ground_truth_data_found:
        print("Unable to find ground truth information for image: {}".format(image_file))

In [ ]:
image_file = "./data/images/2c359fcd7c83a987.jpg"
jsonl_file = "./data/train_annotations.jsonl"

plot_ground_truth_boxes_jsonl(image_file, jsonl_file)

In [ ]:
%pip install azure-ai-ml 

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import json

# Load the config.json file
with open("config.json") as f:
    config = json.load(f)

# Authenticate and create MLClient
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=config["subscription_id"],
    resource_group_name=config["resource_group"],
    workspace_name=config["workspace_name"]
)

print("MLClient created successfully")


In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

my_data = Data(
    path="./data",
    type=AssetTypes.URI_FOLDER,
    description="Coffee Cup Object detection",
    name="coffe-cup-detection",
)

uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)